# Actividad: Entornos Multiagentes

**Integrantes:**
- Karla González Sánchez A01541526
- Luis Ángel Alba Alfaro A01640314
- María Fernanda Elizalde Macías A01634135
- Sofía del Pilar Batiz Martínez A01634125

Modelación de sistemas multiagentes con gráficas computacionales  
Gamaliel Abisay Palomo Briones  
Omar Mendoza Montoya  
Guillermo Gabriel Rivas Aguilar  
14 de noviembre de 2022  

Importar librerías

In [207]:
import agentpy as ap

## Problema 1

## Problema 2

In [208]:
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import random

Definicion del modelo

In [209]:
class ForestModel(ap.Model):

    def setup(self):

        # Create agents (trees)
        n_trees = int(self.p['Tree density'] * (self.p.size**2)) #numero de arboles
        trees = self.agents = ap.AgentList(self, n_trees)

        # Create grid (forest)
        self.forest = ap.Grid(self, [self.p.size]*2, track_empty=True)
        self.forest.add_agents(trees, random=True, empty=True)

        # Initiate a dynamic variable for all trees
        # Condition 0: Alive, 1: Burning, 2: Burned
        self.agents.condition = 0

        # Start a fire from the left side of the grid
        randX = random.randint(1, self.p.size - 1)
        randY = random.randint(1, self.p.size - 1)
        unfortunate_trees = self.forest.agents[randX:randX + 5, randY:randY + 5]
        unfortunate_trees.condition = 1

    def step(self):

        # Select burning trees
        burning_trees = self.agents.select(self.agents.condition == 1)

        # Spread fire
        for tree in burning_trees:
            for neighbor in self.forest.neighbors(tree):
                if neighbor.condition == 0:
                    neighbor.condition = 1 # Neighbor starts burning
            tree.condition = 2 # Tree burns out

        # Stop simulation if no fire is left
        if len(burning_trees) == 0:
            self.stop()

    def end(self):

        # Document a measure at the end of the simulation
        burned_trees = len(self.agents.select(self.agents.condition == 2))
        self.report('Percentage of burned trees',
                    burned_trees / len(self.agents))

Animacion

In [210]:
parameters = {
    'Tree density': 0.6, # Percentage of grid covered by trees
    'size': 50, # Height and length of the grid
    'steps': 100,
}

In [211]:
def animation_plot(model, ax):
    attr_grid = model.forest.attr_grid('condition')
    color_dict = {0:'#7FC97F', 1:'#d62c2c', 2:'#e5e5e5', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a forest fire\n"
                 f"Time-step: {model.t}, Trees left: "
                 f"{len(model.agents.select(model.agents.condition == 0))}")

fig, ax = plt.subplots()
model = ForestModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))

## Problema 3

In [ ]:
import agentpy as ap
import random
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

#### Definición del modelo

In [ ]:
class VacuumModel(ap.Model):

    def setup(self):
        
        # Create agents (vacuum cleaner)
        trashAmount = 1000

        self.robot = ap.AgentList(self, 1)
        self.trash = ap.AgentList(self, trashAmount)

        # Create grid (area to be cleaned)
        m = 50
        n = 60
        gridShape = [m, n]